# preparation

In [1]:
import pyhere
root = pyhere.here().resolve()

print(root)

/work/Walmart_analysis


In [2]:
import kaggle

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# data processing

## data extraccion

In [4]:
!kaggle datasets download -d aslanahmedov/walmart-sales-forecast -p {root.joinpath('data')} --unzip

  0%|                                               | 0.00/3.36M [00:00<?, ?B/s]
100%|███████████████████████████████████████| 3.36M/3.36M [00:00<00:00, 153MB/s]


In [50]:
features = pd.read_csv(root.joinpath('data','features.csv'))
features.head(2)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True


In [6]:
stores = pd.read_csv(root.joinpath('data','stores.csv'))
stores.head(2)

,Store,Type,Size
0,1,A,151315
1,2,A,202307


## data cleaning

In [12]:
stores['Store'] = stores['Store'].astype('object')
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     object
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.2+ KB


In [51]:
features['Store'] = features['Store'].astype('object')
features['Date'] = features['Date'].apply(pd.to_datetime)
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         8190 non-null   object        
 1   Date          8190 non-null   datetime64[ns]
 2   Temperature   8190 non-null   float64       
 3   Fuel_Price    8190 non-null   float64       
 4   MarkDown1     4032 non-null   float64       
 5   MarkDown2     2921 non-null   float64       
 6   MarkDown3     3613 non-null   float64       
 7   MarkDown4     3464 non-null   float64       
 8   MarkDown5     4050 non-null   float64       
 9   CPI           7605 non-null   float64       
 10  Unemployment  7605 non-null   float64       
 11  IsHoliday     8190 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(9), object(1)
memory usage: 712.0+ KB


In [57]:
features_filter = features.copy(deep=True)

filter_null = features_filter.apply(lambda row : False if row.isnull().sum() >= 5 else True,axis=1)
#filter the rows with more or 5 null values
features_filter = features_filter[filter_null]

for column in ['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','CPI','Unemployment']:
    mean_value = features_filter[column].mean()
    features_filter[column] = features_filter[column].fillna(mean_value)

features_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4022 entries, 92 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         4022 non-null   object        
 1   Date          4022 non-null   datetime64[ns]
 2   Temperature   4022 non-null   float64       
 3   Fuel_Price    4022 non-null   float64       
 4   MarkDown1     4022 non-null   float64       
 5   MarkDown2     4022 non-null   float64       
 6   MarkDown3     4022 non-null   float64       
 7   MarkDown4     4022 non-null   float64       
 8   MarkDown5     4022 non-null   float64       
 9   CPI           4022 non-null   float64       
 10  Unemployment  4022 non-null   float64       
 11  IsHoliday     4022 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(9), object(1)
memory usage: 381.0+ KB


In [62]:
features_filed = features.copy(deep=True)

for column in ['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','CPI','Unemployment']:
    mean_value = features_filed[column].mean()
    features_filed[column] = features_filed[column].fillna(mean_value)

features_filed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         8190 non-null   object        
 1   Date          8190 non-null   datetime64[ns]
 2   Temperature   8190 non-null   float64       
 3   Fuel_Price    8190 non-null   float64       
 4   MarkDown1     8190 non-null   float64       
 5   MarkDown2     8190 non-null   float64       
 6   MarkDown3     8190 non-null   float64       
 7   MarkDown4     8190 non-null   float64       
 8   MarkDown5     8190 non-null   float64       
 9   CPI           8190 non-null   float64       
 10  Unemployment  8190 non-null   float64       
 11  IsHoliday     8190 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(9), object(1)
memory usage: 712.0+ KB


In [64]:
df_filter = pd.merge(features_filter,stores,on='Store')
df_filter.head(2)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size
0,1,2011-11-11,59.11,3.297,10382.90,6115.67,215.07,2406.62,6551.42,217.998085,7.866,False,A,151315
1,1,2011-11-18,62.25,3.308,6074.12,254.39,51.98,427.39,5988.57,218.220509,7.866,False,A,151315


In [69]:
df_filed = pd.merge(features_filed,stores,on='Store')
df_filed.head(2)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size
0,1,2010-02-05,42.31,2.572,7032.371786,3384.176594,1760.10018,3292.935886,4132.216422,211.096358,8.106,False,A,151315
1,1,2010-02-12,38.51,2.548,7032.371786,3384.176594,1760.10018,3292.935886,4132.216422,211.242170,8.106,True,A,151315


In [63]:
df_raw = pd.merge(features,stores,on='Store')
df_raw.head(2)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,A,151315


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=740c3fbb-6503-48b5-8790-fc69ebaa42e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>